In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import random
import time
import threading

#import hand module from mediapipe
mp_hands = mp.solutions.hands 
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7)
#import drawing_utils to allow drawing on the images
mp_drawing = mp.solutions.drawing_utils
#bool to stop the counter when camera window is closed
stop_event = threading.Event()
cap = None

global player_gesture
player_gesture = "Unknown"

score_player = 0
score_computer = 0

def calculate_distance(landmark1, landmark2):
    return np.sqrt((landmark1[0] - landmark2[0]) ** 2 + (landmark1[1] - landmark2[1]) ** 2)

def classify_gesture(landmarks):
    landmarks = np.array([(lm.x, lm.y) for lm in landmarks])
        
    #define the tip and base IDs for the fingers
    tip_ids = [4, 8, 12, 16, 20]
    base_ids = [5, 9, 13, 17]
    
    #get the coordinates of the thumb tip and index finger base
    thumb_tip = landmarks[tip_ids[0]]
    index_base = landmarks[base_ids[0]]
    
    #calculate distance between thumb tip and index base for rock
    thumb_tip_to_index_base_distance = calculate_distance(thumb_tip, index_base)
    
    #calculate distances for paper
    index_tip_to_thumb_base_distance = calculate_distance(landmarks[tip_ids[1]], landmarks[0])
    middle_tip_to_middle_base_distance = calculate_distance(landmarks[tip_ids[2]], landmarks[9])
    ring_tip_to_ring_base_distance = calculate_distance(landmarks[tip_ids[3]], landmarks[13])
    pinky_tip_to_pinky_base_distance = calculate_distance(landmarks[tip_ids[4]], landmarks[17])
    
    #check conditions for rock
    if thumb_tip_to_index_base_distance < 0.2 and \
       all(calculate_distance(landmarks[tip_ids[i]], landmarks[base_ids[i-1]]) < 0.1 for i in range(1, 5)):
        return 'Rock'
    
    #check conditions for scissors
    if calculate_distance(thumb_tip, landmarks[tip_ids[3]]) < 0.2 and \
       calculate_distance(thumb_tip, landmarks[tip_ids[4]]) < 0.2 and \
       index_tip_to_thumb_base_distance > 0.15 and \
       middle_tip_to_middle_base_distance > 0.15:
        return 'Scissors'
    
    #check conditions for paper
    if index_tip_to_thumb_base_distance > 0.1 and \
       middle_tip_to_middle_base_distance > 0.1 and \
       ring_tip_to_ring_base_distance > 0.1 and \
       pinky_tip_to_pinky_base_distance > 0.1:
        return 'Paper'

def refresh_camera():
    
    global player_gesture
    global score_player
    global score_computer
    global stop_event
    #access the webcam
    global cap
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        #ret is a boolean that indicates if capture is successful and frame is the image
        ret, frame = cap.read()
        if not ret:
            break
        #flip the image horizontally because the webcam image is inverted
        frame = cv2.flip(frame, 1)
        #convert BGR image (used by cv2) to RBG image (used by mediapipe)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #return results of detection. These results can include the position of the different landmarks, the confidence scores,...
        results = hands.process(frame_rgb)
        player_gesture = "Unknown"
    
    #'results.multi_hand_landmarks' displays the coordinates of the detected hand key points (21 points).
    #Each key point is represented by a set of three-dimensional coordinates in image space.
    #The output looks like a list of dictionaries, where each dictionary represents a detected
    #hand and contains the coordinates of its keypoints. If no hand is detected, it returns None
    
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                #draw landmarks
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                #classify gesture based on landmarks
                player_gesture = classify_gesture(hand_landmarks.landmark) #hand_landmarks.landmark returns an array of 21 elements with x,y,z coordinates for each one of them
                
                #display
                cv2.putText(frame, f'Player: {player_gesture}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, f'Score of the player: {score_player}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, f'Score of the computer: {score_computer}', (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Gesture Recognition", frame)
    
        #press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_event.set()
            print("Stop!")
            print("Final Score : Computer {} - {} You".format(score_computer, score_player))
            if (score_player> score_computer):
                print(" You won the match")
            elif (score_computer>score_player):
                print("You lost the match")
            else:
                print("This match is a draw")
            break
    
    #when everything is done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    
def computer_choice():
    #define a list of possible computer gestures
    computer_options = ['Rock', 'Paper', 'Scissors']
    computer_choice = random.choice(computer_options)
    return computer_choice

def round_outcome():
    global comp_choice
    global score_computer
    global score_player   
    
    if(comp_choice == 'Rock'):
        if(player_gesture == 'Rock'):
            print('Tie')
        elif(player_gesture == 'Paper'):
            print('Win')
            score_player += 1
        elif(player_gesture == 'Scissors'):
            print('Lost')
            score_computer += 1
        else:
            print("Unknown, Let's replay this one")
                                
    elif(comp_choice == 'Paper'):
        if(player_gesture == 'Rock'):
            print('Lost')
            score_computer += 1
        elif(player_gesture == 'Paper'):
            print('Tie')
        elif(player_gesture == 'Scissors'):
            print('Win')
            score_player += 1
        else:
            print("Unknown, Let's replay this one")
                                
    else:
        if(player_gesture == 'Rock'):
            print('Win')
            score_player += 1
        elif(player_gesture == 'Paper'):
            print('Lost')
            score_computer += 1
        elif(player_gesture == 'Scissors'):
            print('Tie')
        else:
            print("Unknown, Let's replay this one")

def timer_function():
    global cap
    global score_player
    global score_computer
    global comp_choice
    
    i = 5
    while True:
        if stop_event.is_set():
            break

        if (cap!= None):
            if (cap.isOpened()):
                print(i)
                time.sleep(1)
                i-=1
                if(i==0):            
                    print("Timer expired!")
                    i = 5
                    comp_choice = computer_choice()
                    print(player_gesture)
                    round_outcome()
                
            

#create and start the camera refresh thread
camera_thread = threading.Thread(target=refresh_camera)
camera_thread.start()

#create and start the timer thread
timer_thread = threading.Thread(target=timer_function)
timer_thread.start()

#wait for both threads to finish
camera_thread.join()
timer_thread.join()
